In [1]:
#Apache Spark : Genel amaçlı paralel veri işleme frameworku
#Hadoop üzerinde çalışabilir
#Büyük Veri İşleme, Büyük Veri ile Makine Öğrenmesi ===> HADOOP + Spark
!apt-get update
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
!update-alternatives --set java /usr/lib/jvm/java-8-openjdk-amd64/jre/bin/java
!java -version
!pip install pyspark

Get:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Ign:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Ign:3 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:4 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:5 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:6 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Hit:7 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:8 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Get:9 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:11 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Get:13 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Hit:14 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Hit:15 http://ppa

In [4]:
from geopy.distance import geodesic
from geopy.point import Point
from geopy.geocoders import Nominatim
from operator import add
from pyspark import SparkContext

sc = SparkContext.getOrCreate()

capitals = sc.textFile("Capitals.txt")


#This function is created with the purpose of splitting data
def splitTake(line):
    arr = line.split('\t')
    ulke=arr[1]
    sehır=arr[2]
    latitude=arr[3].replace(',', ".")
    longtitude =arr[4].replace(',', ".")
    arr=line.split(',')
    return ulke,sehır,float(latitude),float(longtitude)

capitals_correct = capitals.map(lambda line : splitTake(line)).filter(lambda x: x[2]<90 and x[2]>-90) 
capitals_correction = capitals.map(lambda line : splitTake(line)).filter(lambda x: x[2]>90 or x[2]<-90)
#There is some of the values of latitude is miscalculated, so I prefer to divide them correct and incorrect.
#By doing it, ı can take the original latitude and longtitude of them from geocoder.
def loc_bul(data):
  geo = Nominatim(user_agent="my_user_agent")
  city = data[1]
  country = data[0]
  loc = geo.geocode(city+','+ country)
  return country,city,float(loc.latitude),float(loc.longitude)

capitals_correction = capitals_correction.map(lambda line : loc_bul(line))

#union correct and inccorrect data 
capitals_union = capitals_correct.union(capitals_correction)

#cartesian join
capitals_cartesian = capitals_union.cartesian(capitals_union)



def distance(data):
  country_1 = data[0][0]
  city_1 = data[0][1]
  lat_1 = data[0][2]
  lon_1 = data[0][3]
  country_2 = data[1][0]
  city_2 = data[1][1]
  lat_2 = data[1][2]
  lon_2 = data[1][3]
  distance = geodesic((lat_1,lon_1),(lat_2,lon_2)).miles
  return (city_1,city_2,int(distance))


# After using cartesian join, there was a problem data multiplication problem. 
#In order to solve this problem I used .zipWithIndex() method and divide them 2.
#I solved multiplication problem of cartesian join
result = capitals_cartesian.map(lambda line: distance(line)).filter(lambda x: x[0] != x[1]).sortBy(lambda x: x[2],ascending=False).zipWithIndex()


result = result.filter(lambda x: x[1]%2 == 0)
result.take(2)



[(('Abuja ', 'Nukunonu ', 12401), 0), (('Asunción ', 'Taipei ', 12388), 2)]